In [1]:
!nvidia-smi

Wed Sep 20 20:51:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.296 --progress-bar off
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install unstructured==0.10.16 > /dev/null

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.33.2 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.14.0 which is incompatible.


In [3]:
!gdown 1aTtq5rgUseJrFVxNNthonAuf9CrI6JI6

Downloading...
From: https://drive.google.com/uc?id=1aTtq5rgUseJrFVxNNthonAuf9CrI6JI6
To: /content/bitcoin.md
100% 21.9k/21.9k [00:00<00:00, 65.9MB/s]


In [4]:
import torch

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "TheBloke/Llama-2-13b-Chat-GPTQ"

In [5]:
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [6]:
%%time
result = llm(
    "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
)

CPU times: user 8.22 s, sys: 378 ms, total: 8.6 s
Wall time: 11.9 s


In [7]:
print(result)



Answer: Sure! Here's the difference between ChatGPT and open-source large language models (LLMs) in two lines:

ChatGPT is a proprietary, closed-source AI model developed by Meta AI that offers a more user-friendly interface and seamless integration with other Meta products, while open-source LLMs like BERT and RoBERTa are freely available for anyone to use and modify, but may require more technical expertise to integrate into applications.


## Prompt Template

In [8]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [9]:
text = "Explain what are Deep Neural Networks in 2-3 sentences"

In [10]:
print(prompt.format(text=text))


<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>

Explain what are Deep Neural Networks in 2-3 sentences [/INST]



In [11]:
%%time
result = llm(prompt.format(text=text))

CPU times: user 5.85 s, sys: 26.2 ms, total: 5.87 s
Wall time: 5.93 s


In [12]:
print(result)


Hey there, young minds! So, you wanna know about Deep Neural Networks? Well, imagine you have a super powerful computer that can learn and make decisions all on its own, kinda like how your brain works! Deep Neural Networks are like a bunch of these computers working together to solve really tough problems, like recognizing pictures or understanding speech. They're like the ultimate team players, and they're changing the game in fields like self-driving cars, medical diagnosis, and more!


## Create Chain

In [13]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

In [14]:
%%time
result = chain.run(text)

CPU times: user 5.59 s, sys: 61.8 ms, total: 5.66 s
Wall time: 5.67 s


In [15]:
print(result)


Hey there, young minds! So, you wanna know about Deep Neural Networks? Well, imagine you have a super powerful computer that can learn and make decisions all on its own, kinda like how your brain works! Deep Neural Networks are like a bunch of these computers working together to solve really tough problems, like recognizing pictures or understanding speech. They're like the ultimate team players, and they're changing the game in fields like self-driving cars, medical diagnosis, and more!


## Chaining Chains

In [16]:
template = "<s>[INST] Use the summary {summary} and give 3 examples of practical applications with 1 sentence explaining each [/INST]"

examples_prompt = PromptTemplate(
    input_variables=["summary"],
    template=template,
)
examples_chain = LLMChain(llm=llm, prompt=examples_prompt)

In [17]:
from langchain.chains import SimpleSequentialChain

multi_chain = SimpleSequentialChain(chains=[chain, examples_chain], verbose=True)

In [18]:
result = multi_chain.run(text)



> Entering new SimpleSequentialChain chain...

Hey there, young minds! So, you wanna know about Deep Neural Networks? Well, imagine you have a super powerful computer that can learn and make decisions all on its own, kinda like how your brain works! Deep Neural Networks are like a bunch of these computers working together to solve really tough problems, like recognizing pictures or understanding speech. They're like the ultimate team players, and they're changing the game in fields like self-driving cars, medical diagnosis, and more!
  Sure thing! Here are three examples of practical applications of Deep Neural Networks:

1. Self-Driving Cars: Deep Neural Networks can be used to train autonomous vehicles to recognize objects on the road, such as pedestrians, other cars, and traffic lights, allowing them to make safe and efficient decisions.
2. Medical Diagnosis: Deep Neural Networks can be trained on large datasets of medical images and patient data to help doctors diagnose diseases 

In [19]:
print(result.strip())

Sure thing! Here are three examples of practical applications of Deep Neural Networks:

1. Self-Driving Cars: Deep Neural Networks can be used to train autonomous vehicles to recognize objects on the road, such as pedestrians, other cars, and traffic lights, allowing them to make safe and efficient decisions.
2. Medical Diagnosis: Deep Neural Networks can be trained on large datasets of medical images and patient data to help doctors diagnose diseases and conditions more accurately and efficiently than ever before.
3. Speech Recognition: Deep Neural Networks can be used to improve speech recognition systems, enabling devices like smartphones and virtual assistants to better understand and respond to voice commands.


## Embeddings

In [20]:
from langchain.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader("bitcoin.md")
docs = loader.load()
len(docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


1

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

29

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

In [23]:
query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

1024


In [24]:
%%time
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 2.16 s, sys: 29.4 ms, total: 2.19 s
Wall time: 2.48 s


In [25]:
results = db.similarity_search("proof-of-work majority decision making", k=2)
len(results)

2

In [26]:
print(results[0].page_content)

The proof-of-work also solves the problem of determining representation
in majority decision making. If the majority were based on
one-IP-address-one-vote, it could be subverted by anyone able to
allocate many IPs. Proof-of-work is essentially one-CPU-one-vote. The
majority decision is represented by the longest chain, which has the
greatest proof-of-work effort invested in it. If a majority of CPU power
is controlled by honest nodes, the honest chain will grow the fastest
and outpace any competing chains. To modify a past block, an attacker
would have to redo the proof-of-work of the block and all blocks after
it and then catch up with and surpass the work of the honest nodes. We
will show later that the probability of a slower attacker catching up
diminishes exponentially as subsequent blocks are added.


In [27]:
template = """
<s>[INST] <<SYS>>
Act as a cryptocurrency expert. Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""

In [28]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [29]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [32]:
%%time
result = qa_chain(
    "How does proof-of-work solves the majority decision making problem? Explain like I am five."
)
result

CPU times: user 18.7 s, sys: 58.1 ms, total: 18.8 s
Wall time: 27.1 s


{'query': 'How does proof-of-work solves the majority decision making problem? Explain like I am five.',
 'result': "\nOkay, little buddy! So you know how there are lots of different people who want to make decisions about things like what games to play or what food to eat? Well, sometimes these people might not agree on what they want, and that's where proof-of-work comes in!\n\nProof-of-work is like a special kind of vote that shows how much work someone did. It's like if you had to do a puzzle before you could say what game you wanted to play. The person who does the most work gets to choose what game everyone plays!\n\nBut here's the cool thing about proof-of-work: it makes sure that only the people who really want to play the game get to choose. If someone tries to cheat and say they did more work than they actually did, the other kids won't believe them because they can see how much work was really done.\n\nSo, when we use proof-of-work to make decisions, we can be sure that the 

In [31]:
print(result["result"].strip())

Okay, little buddy! So you know how there are lots of different people who want to make decisions about things like what games to play or what food to eat? Well, sometimes these people might not agree on what they want, and that's where proof-of-work comes in!

Proof-of-work is like a special kind of vote that shows how much work someone did. It's like if you had to do a puzzle before you could say what game you wanted to play. The person who does the most work gets to choose what game everyone plays!

But here's the cool thing about proof-of-work: it makes sure that only the people who really want to play the game get to choose. If someone tries to cheat and say they did more work than they actually did, the other kids won't believe them because they can see how much work was really done.

So, when we use proof-of-work to make decisions, we can be sure that the person who chooses the game is the one who really wants to play it, and not just someone who wants to cheat and pick their fa

In [44]:
%%time
result = qa_chain(
    "Summarize the privacy compared to the traditional banking model in 2-3 sentences."
)
result

CPU times: user 8.65 s, sys: 257 ms, total: 8.9 s
Wall time: 9.27 s


{'query': 'Summarize the privacy compared to the traditional banking model in 2-3 sentences.',
 'result': '\nIn contrast to the traditional banking model, which relies on limited access to information to maintain privacy, cryptocurrencies like Bitcoin provide greater privacy by keeping public keys anonymous, allowing individuals to send and receive funds without revealing their identities. This is similar to the level of information released by stock exchanges, where the time and size of individual trades are made public, but without telling who the parties were. Additionally, cryptocurrencies use decentralized networks and encryption techniques to protect user data and prevent unauthorized access, further enhancing privacy compared to traditional banking systems.',
 'source_documents': [Document(page_content='Privacy\n\nThe traditional banking model achieves a level of privacy by limiting\naccess to information to the parties involved and the trusted third\nparty. The necessity to ann

In [59]:
from textwrap import fill

print(fill(result["result"].strip(), width=80))

In contrast to the traditional banking model, which relies on limited access to
information to maintain privacy, cryptocurrencies like Bitcoin provide greater
privacy by keeping public keys anonymous, allowing individuals to send and
receive funds without revealing their identities. This is similar to the level
of information released by stock exchanges, where the time and size of
individual trades are made public, but without telling who the parties were.
Additionally, cryptocurrencies use decentralized networks and encryption
techniques to protect user data and prevent unauthorized access, further
enhancing privacy compared to traditional banking systems.
